In [1]:
import os
from langchain_community.llms import ollama
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [2]:
def retres(llm,temp,ref,tar,span,ser,cat):
    chain = temp | llm | StrOutputParser()    
    response = chain.invoke({"ref": ref, "tar": tar,"error_span":span,"error_category":cat,"severity":ser}) 
    return response

In [3]:
import pandas as pd
df=pd.read_csv('dataset.csv')
df=df[:20]

In [4]:
example=[]
for i in range(0,11):
    a={"ref":df['Ref'][i],"tar":df['Tar'][i],"error_span":df['error_span'][i],"severity":df['severity'][i],"error_category":df['error_cat'][i]}
    example.append(a)

In [5]:
example

[{'ref': 'A cat sat on a mat.',
  'tar': 'A mat sat on a cat.',
  'error_span': '"mat sat on a cat"',
  'severity': 'major',
  'error_category': 'Unnatural'},
 {'ref': 'The mother sang a lullaby to the sleepy cat',
  'tar': 'The chair sang a lullaby to the sleepy cat',
  'error_span': '"chair"',
  'severity': 'major',
  'error_category': 'Unnatural'},
 {'ref': 'Break a leg',
  'tar': 'Break one of your legs',
  'error_span': '"one of your legs"',
  'severity': 'major',
  'error_category': 'Mistranslation, Addition'},
 {'ref': 'Please make yourself at home',
  'tar': 'Please create your home',
  'error_span': '"create your home"',
  'severity': 'major',
  'error_category': 'Mistranslation'},
 {'ref': 'The software is buggy',
  'tar': 'The software has insects',
  'error_span': '"software has insects"',
  'severity': 'major',
  'error_category': 'Mistranslation'},
 {'ref': 'Although it was raining, she went for a walk',
  'tar': 'Although it is raining, she went for a walk',
  'error_spa

In [6]:

template = """
Reference sentence: {ref}
Target sentence: {tar}
Error Span: {error_span}
Error Category {error_category}
\
"""

temp = PromptTemplate(
    input_variables=["ref", "tar","error_span","severity","error_category"],
    template=template,
)
fewshotprompt=FewShotPromptTemplate(
    examples=example,
    example_prompt=temp,
    input_variables=["ref", "tar","error_span","severity","error_category"],
    suffix="Reference Sentence:{ref}\nTarget Sentence:{tar}\nError Span:{error_span}\nError Category {error_category}",
    prefix="Classify the error category as one of the following categories: Mistranslation, Incorrect Word Choice, Contextual Error, Addition in the target sentence with respect to the given reference sentence and severity (in terms of minor, major) . Generate error category error span and severity in json format."
)


## LLAMA2

In [13]:
for i in range(0,len(df)):
    print('Sentence '+str(i)+'\n')
    print('Reference sentence '+df['Ref'][i]+'\n')
    print('Target sentence '+df['Tar'][i]+'\n')
    print(retres(ollama.Ollama(temperature=0,model='llama2'),fewshotprompt,df['Ref'][i],df['Tar'][i],df['error_span'][i],df['severity'][i],df['error_cat'][i]))
    print('\n\n')

Sentence 0

Reference sentence A cat sat on a mat.

Target sentence A mat sat on a cat.

Here are the errors in the target sentences and their corresponding categories:

1. Target sentence: A mat sat on a cat.
Error category: Unnatural
Error span: "mat sat on a cat"
2. Target sentence: The chair sang a lullaby to the sleepy cat.
Error category: Unnatural
Error span: "chair"
3. Target sentence: Break one of your legs.
Error category: Mistranslation, Addition
Error span: "one of your legs"
4. Target sentence: Please create your home.
Error category: Mistranslation
Error span: "create your home"
5. Target sentence: The software has insects.
Error category: Mistranslation
Error span: "software has insects"
6. Target sentence: Although it is raining, she went for a walk.
Error category: Grammatical error
Error span: "it is raining, she went for a walk"
7. Target sentence: I found the buyer of the wrong order before.
Error category: Grammatical error
Error span: "of the wrong order before"
8

Here are the errors and their corresponding categories in JSON format:

[
{
"error_span": "mat sat on a cat",
"error_category": "unnatural"
},
{
"error_span": "chair",
"error_category": "unnatural"
},
{
"error_span": "one of your legs",
"error_category": "mistranslation, addition"
},
{
"error_span": "create your home",
"error_category": "mistranslation"
},
{
"error_span": "software has insects",
"error_category": "mistranslation"
},
{
"error_span": "it is raining, she went for a walk",
"error_category": "grammatical error"
},
{
"error_span": "of the wrong order before",
"error_category": "grammatical error"
},
{
"error_span": "they has been",
"error_category": "grammatical error"
},
{
"error_span": "tomorrow",
"error_category": "omission"
},
{
"error_span": "ahead and after the schedule",
"error_category": "addition, omission"
},
{
"error_span": "waked up",
"error_category": "spelling & typological"
}
]



Sentence 6

Reference sentence I found the buyer with the wrong order.

Target s

Here are the errors in the target sentences and their corresponding categories:

1. Error Category: Unnatural
Error Span: "mat sat on a cat"
Reference Sentence: A cat sat on a mat.
Target Sentence: A mat sat on a cat.
2. Error Category: Unnatural
Error Span: "chair"
Reference Sentence: The mother sang a lullaby to the sleepy cat
Target Sentence: The chair sang a lullaby to the sleepy cat.
3. Error Category: Mistranslation, Addition
Error Span: "one of your legs"
Reference Sentence: Break a leg
Target Sentence: Break one of your legs.
4. Error Category: Mistranslation
Error Span: "create your home"
Reference Sentence: Please make yourself at home
Target Sentence: Please create your home.
5. Error Category: Mistranslation
Error Span: "software has insects"
Reference Sentence: The software is buggy
Target Sentence: The software has insects.
6. Error Category: Grammatical error
Error Span: "it is raining, she went for a walk"
Reference Sentence: Although it was raining, she went for a walk

Here are the errors and their corresponding categories in JSON format:

[
{
"error_span": "mat sat on a cat",
"error_category": "unnatural"
},
{
"error_span": "chair",
"error_category": "unnatural"
},
{
"error_span": "one of your legs",
"error_category": "mistranslation, addition"
},
{
"error_span": "create your home",
"error_category": "mistranslation"
},
{
"error_span": "software has insects",
"error_category": "mistranslation"
},
{
"error_span": "it is raining, she went for a walk",
"error_category": "grammatical error"
},
{
"error_span": "of the wrong order before",
"error_category": "grammatical error"
},
{
"error_span": "they has been studying French for three years",
"error_category": "grammatical error"
},
{
"error_span": "tomorrow",
"error_category": "omission"
},
{
"error_span": "ahead and after the schedule",
"error_category": "addition, omission"
},
{
"error_span": "waked up",
"error_category": "spelling & typological"
}
]



Sentence 16

Reference sentence It has been 1 ho

## LLAMA3

In [14]:
for i in range(0,len(df)):
    print('Sentence '+str(i)+'\n')
    print('Reference sentence '+df['Ref'][i]+'\n')
    print('Target sentence '+df['Tar'][i]+'\n')
    print(retres(ollama.Ollama(temperature=0,model='llama3'),fewshotprompt,df['Ref'][i],df['Tar'][i],df['error_span'][i],df['severity'][i],df['error_cat'][i]))
    print('\n\n')

Sentence 0

Reference sentence A cat sat on a mat.

Target sentence A mat sat on a cat.

Here are the error categories, spans, and severities in JSON format:

```
[
  {
    "error_category": "Unnatural",
    "error_span": "mat sat on a cat",
    "severity": "major"
  },
  {
    "error_category": "Unnatural",
    "error_span": "chair",
    "severity": "minor"
  },
  {
    "error_category": "Mistranslation, Addition",
    "error_span": "one of your legs",
    "severity": "major"
  },
  {
    "error_category": "Mistranslation",
    "error_span": "create your home",
    "severity": "minor"
  },
  {
    "error_category": "Mistranslation",
    "error_span": "software has insects",
    "severity": "major"
  },
  {
    "error_category": "Grammatical error",
    "error_span": "it is raining, she went for a walk",
    "severity": "minor"
  },
  {
    "error_category": "Grammatical error",
    "error_span": "of the wrong order before",
    "severity": "major"
  },
  {
    "error_category": "Gramm

Here are the classifications in JSON format:

```
[
  {
    "reference_sentence": "A cat sat on a mat.",
    "target_sentence": "A mat sat on a cat.",
    "error_span": "mat sat on a cat",
    "error_category": "Unnatural",
    "severity": "major"
  },
  {
    "reference_sentence": "The mother sang a lullaby to the sleepy cat",
    "target_sentence": "The chair sang a lullaby to the sleepy cat",
    "error_span": "chair",
    "error_category": "Unnatural",
    "severity": "major"
  },
  {
    "reference_sentence": "Break a leg",
    "target_sentence": "Break one of your legs",
    "error_span": "one of your legs",
    "error_category": "Mistranslation, Addition",
    "severity": "major"
  },
  {
    "reference_sentence": "Please make yourself at home",
    "target_sentence": "Please create your home",
    "error_span": "create your home",
    "error_category": "Mistranslation",
    "severity": "major"
  },
  {
    "reference_sentence": "The software is buggy",
    "target_sentence": "T

Here are the error categories, spans, and severities in JSON format:

```
[
  {
    "error_category": "Unnatural",
    "error_span": "mat sat on a cat"
  },
  {
    "error_category": "Unnatural",
    "error_span": "chair"
  },
  {
    "error_category": "Mistranslation, Addition",
    "error_span": "one of your legs"
  },
  {
    "error_category": "Mistranslation",
    "error_span": "create your home"
  },
  {
    "error_category": "Mistranslation",
    "error_span": "software has insects"
  },
  {
    "error_category": "Grammatical error",
    "error_span": "it is raining, she went for a walk"
  },
  {
    "error_category": "Grammatical error",
    "error_span": "of the wrong order before"
  },
  {
    "error_category": "Grammatical error",
    "error_span": "they has been"
  },
  {
    "error_category": "Ommision",
    "error_span": "tomorrow"
  },
  {
    "error_category": "Addition, Ommision",
    "error_span": "and after the schedule"
  },
  {
    "error_category": "Spelling & Typo

Here are the error categories, spans, and severities in JSON format:

```
[
  {
    "error_category": "Unnatural",
    "error_span": "mat sat on a cat",
    "severity": "minor"
  },
  {
    "error_category": "Unnatural",
    "error_span": "chair",
    "severity": "minor"
  },
  {
    "error_category": "Mistranslation, Addition",
    "error_span": "one of your legs",
    "severity": "major"
  },
  {
    "error_category": "Mistranslation",
    "error_span": "create your home",
    "severity": "minor"
  },
  {
    "error_category": "Mistranslation",
    "error_span": "software has insects",
    "severity": "minor"
  },
  {
    "error_category": "Grammatical error",
    "error_span": "it is raining, she went for a walk",
    "severity": "major"
  },
  {
    "error_category": "Grammatical error",
    "error_span": "of the wrong order before",
    "severity": "minor"
  },
  {
    "error_category": "Grammatical error",
    "error_span": "they has been",
    "severity": "major"
  },
  {
    "e

Here are the error categories in JSON format:

```
[
  {
    "error_category": "Unnatural",
    "error_span": "mat sat on a cat"
  },
  {
    "error_category": "Unnatural",
    "error_span": "chair"
  },
  {
    "error_category": "Mistranslation, Addition",
    "error_span": "one of your legs"
  },
  {
    "error_category": "Mistranslation",
    "error_span": "create your home"
  },
  {
    "error_category": "Mistranslation",
    "error_span": "software has insects"
  },
  {
    "error_category": "Grammatical error",
    "error_span": "it is raining, she went for a walk"
  },
  {
    "error_category": "Grammatical error",
    "error_span": "of the wrong order before"
  },
  {
    "error_category": "Grammatical error",
    "error_span": "they has been"
  },
  {
    "error_category": "Ommision",
    "error_span": "tomorrow"
  },
  {
    "error_category": "Addition, Ommision",
    "error_span": "and after the schedule"
  },
  {
    "error_category": "Spelling & Typological",
    "error_sp

## Mistral

In [15]:
for i in range(0,len(df)):
    print('Sentence '+str(i)+'\n')
    print('Reference sentence '+df['Ref'][i]+'\n')
    print('Target sentence '+df['Tar'][i]+'\n')
    print(retres(ollama.Ollama(temperature=0,model='mistral'),fewshotprompt,df['Ref'][i],df['Tar'][i],df['error_span'][i],df['severity'][i],df['error_cat'][i]))
    print('\n\n')

Sentence 0

Reference sentence A cat sat on a mat.

Target sentence A mat sat on a cat.

 {
  "Error Span": "mat sat on a cat",
  "Error Category": "Unnatural",
  "Severity": "Major"
}

{
  "Error Span": "chair",
  "Error Category": "Unnatural",
  "Severity": "Minor"
}

{
  "Error Span": "one of your legs",
  "Error Category": ["Mistranslation", "Addition"],
  "Severity": "Major"
}

{
  "Error Span": "create your home",
  "Error Category": "Mistranslation",
  "Severity": "Major"
}

{
  "Error Span": "software has insects",
  "Error Category": "Mistranslation",
  "Severity": "Major"
}

{
  "Error Span": "it is raining, she went for a walk",
  "Error Category": "Grammatical error",
  "Severity": "Minor"
}

{
  "Error Span": "of the wrong order before",
  "Error Category": "Grammatical error",
  "Severity": "Major"
}

{
  "Error Span": "they has been",
  "Error Category": "Grammatical error",
  "Severity": "Minor"
}

{
  "Error Span": "tomorrow",
  "Error Category": "Ommission",
  "Severi

 The corrected sentence is: "It's been an hour, and it hasn't been delivered yet."

The error span was "an arrived" which should be "an hour" and "hasn't arrived" which should be "hasn't been delivered". The error category is Grammatical error, Incorrect word choice.



Sentence 17

Reference sentence Then I will use my points to re-place the order, thank you.

Target sentence Then I'll re order with points. Thank you

 To correct the error, we can replace "re order" with "re-place". Here is the corrected sentence:

Target Sentence: Then I'll re-place the order with points. Thank you
Error Span: "re-place"
Error Category: Correction



Sentence 18

Reference sentence If you cannot change the location of the branch store for me, can I change it to delivery?

Target sentence If you can't change the location of the branch for me, can you change it to delivery?

 The error in this sentence is due to inconsistency in verb tense usage. The subject "you" is using present tense ("can't"), whil

## TinyDolphin

In [16]:
for i in range(0,len(df)):
    print('Sentence '+str(i)+'\n')
    print('Reference sentence '+df['Ref'][i]+'\n')
    print('Target sentence '+df['Tar'][i]+'\n')
    print(retres(ollama.Ollama(temperature=0,model='tinydolphin'),fewshotprompt,df['Ref'][i],df['Tar'][i],df['error_span'][i],df['severity'][i],df['error_cat'][i]))
    print('\n\n')


Sentence 0

Reference sentence A cat sat on a mat.

Target sentence A mat sat on a cat.

 {
  "Category": "Mistranslation",
  "Severity": "Major",
  "ErrorSpan": ["cat sat on a mat.", "mat sat on a cat."],
  "ErrorCategory": ["Unnatural"]
}<|im_end|>



Sentence 1

Reference sentence The mother sang a lullaby to the sleepy cat

Target sentence The chair sang a lullaby to the sleepy cat

 The sentence should be "the chair sang a lullaby to the sleepy cat."

In this case, the error in the sentence is that it's grammatically incorrect. The correct sentence structure would be:

"The chair sang a lullaby to the sleepy cat."

Here are some more examples of grammatically correct sentences:

1. "I love you" - This is a simple, declarative sentence.
2. "He loves her" - This is an interrogative sentence.
3. "The cat sat on the chair" - This is a compound sentence with two independent clauses.
4. "The cat sat on the chair" - This is a compound sentence with one independent clause and one dependen

 The error message suggests that there is an inconsistency in the way you are trying to change the location of the branch. It seems like you might have made a typo or gotten confused about what you meant. Here's how you can fix it:

"If you cannot change the location of the branch for me, can I change it to delivery?"

This sentence should be grammatically correct and make sense. If you want to change the location of the branch to delivery, you can say:

"If you can't change the location of the branch for me, can I change it to delivery?"<|im_end|>



Sentence 19

Reference sentence I’ve waited so long for you.

Target sentence Waiting for you so long

 The sentence is grammatically correct, but it has a typo in the word "waiting." It should be "waiting for."<|im_end|>





## Gemma:2b

In [7]:
for i in range(0,len(df)):
    print('Sentence '+str(i)+'\n')
    print('Reference sentence '+df['Ref'][i]+'\n')
    print('Target sentence '+df['Tar'][i]+'\n')
    print(retres(ollama.Ollama(temperature=0,model='gemma:2b'),fewshotprompt,df['Ref'][i],df['Tar'][i],df['error_span'][i],df['severity'][i],df['error_cat'][i]))
    print('\n\n')

Sentence 0

Reference sentence A cat sat on a mat.

Target sentence A mat sat on a cat.

{
  "error_category": "Unnatural",
  "error_span": "mat sat on a cat",
  "severity": "Major"
}

{
  "error_category": "Unnatural",
  "error_span": "chair",
  "severity": "Major"
}

{
  "error_category": "Mistranslation",
  "error_span": "one of your legs",
  "severity": "Minor"
}

{
  "error_category": "Mistranslation",
  "error_span": "although it was raining, she went for a walk",
  "severity": "Minor"
}

{
  "error_category": "Grammatical error",
  "error_span": "they has been",
  "severity": "Minor"
}

{
  "error_category": "Grammatical error",
  "error_span": "She will arrive at the airport at noon tomorrow",
  "severity": "Minor"
}

{
  "error_category": "Ommision",
  "error_span": "We have completed the project ahead and after the schedule",
  "severity": "Minor"
}

{
  "error_category": "Spelling & Typological",
  "error_span": "A cat sat on a mat.",
  "severity": "Minor"
}



Sentence 1

R

{
  "error_category": "Unnatural",
  "error_span": "mat sat on a cat",
  "severity": "Major"
},
{
  "error_category": "Unnatural",
  "error_span": "chair",
  "severity": "Major"
},
{
  "error_category": "Mistranslation",
  "error_span": "one of your legs",
  "severity": "Minor"
},
{
  "error_category": "Mistranslation",
  "error_span": "although it was raining",
  "severity": "Minor"
},
{
  "error_category": "Grammatical error",
  "error_span": "they has been",
  "severity": "Minor"
},
{
  "error_category": "Grammatical error",
  "error_span": "She will arrive at the airport at noon tomorrow",
  "severity": "Minor"
},
{
  "error_category": "Ommision",
  "error_span": "We have completed the project ahead and after the schedule",
  "severity": "Minor"
},
{
  "error_category": "Addition, Ommision",
  "error_span": "She woke up early to catch the train",
  "severity": "Minor"
}




Sentence 10

Reference sentence She woke up early to catch the train

Target sentence She waked up early to c

{
  "error_category": "Unnatural",
  "error_span": "mat sat on a cat",
  "severity": "Major"
},
{
  "error_category": "Unnatural",
  "error_span": "chair",
  "severity": "Major"
},
{
  "error_category": "Mistranslation",
  "error_span": "one of your legs",
  "severity": "Minor"
},
{
  "error_category": "Mistranslation",
  "error_span": "although it was raining",
  "severity": "Minor"
},
{
  "error_category": "Grammatical error",
  "error_span": "they has been",
  "severity": "Minor"
},
{
  "error_category": "Grammatical error",
  "error_span": "She will arrive at the airport at noon tomorrow",
  "severity": "Minor"
},
{
  "error_category": "Ommision",
  "error_span": "We have completed the project ahead of schedule",
  "severity": "Minor"
},
{
  "error_category": "Spelling & Typological",
  "error_span": "Then I'll re order with points, thank you",
  "severity": "Minor"
}



Sentence 18

Reference sentence If you cannot change the location of the branch store for me, can I change it to